# 라이브러리 로드

In [19]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from tqdm.notebook import tqdm

In [2]:
# python0205 에서 저장한 csv 파일 불러오기
df = pd.read_csv("seoul_120_list.csv")
df.shape

(2304, 5)

# 일부 데이터 수집
* 서버에 부담이 될 수 있고, 오래 걸리므로 일부만 수집

In [3]:
# data 일부만 한정
df = df[:10]
df

,번호,제목,생산일,조회수,내용번호
0,2304,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,126,26695536
1,2303,다자녀가정 실내 바닥매트 지원,2021-08-17,894,25670204
2,2302,[서울산업진흥원] 서울메이드란?,2021-06-29,5230,23194045
3,2301,마포 뇌병변장애인 비전센터,2021-03-12,1310,22477798
4,2300,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1626,21212235
5,2299,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1351,21095911
6,2298,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,1997,18955676
7,2297,"[종료사업] S-Taxi 앱 (에스택시, 서울택시승차앱) 시범운영",2019-06-03,2049,17949033
8,2296,장기요양 이동지원서비스(모두타 돌봄택시),2019-05-27,2800,17896364
9,2295,아름다운청년 전태일기념관,2019-03-20,1632,17505766


# 문서 분류 수집

In [4]:
url = "https://opengov.seoul.go.kr/civilappeal/view/?nid=23194045"
# url = "https://opengov.seoul.go.kr/civilappeal/view/?nid=22904492"
response = requests.get(url)
response

<Response [200]>

In [5]:
# html parsing
html = bs(response.text)

In [6]:
# copy.selector
content = html.select("div.view-content.view-content-article > div > div.line-all")[0].get_text()
content

'\n\xa0서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플랫폼 브랜드로 서울이 만들고 세계가 함께 쓰는 브랜드 입니다.\xa0 \xa0 - 서울의 감성을 담은 기업과 협업하여 브랜드 상품을 발굴. \xa0 -\xa0사용을 원하는 중소기업에게 서울메이드 브랜드 제공 등 \xa0 -\xa0 관련홈페이지 :\xa0서울메이드 (https://seoulmade.com/) \xa0※ 상세문의사항 :\xa0서울산업진흥원\xa0브랜드전략팀 02-2222-3764 '

In [7]:
# 문서 정보
table = pd.read_html(response.text)[-1]
table

,0,1,2,3
0,원본시스템,다산콜센터,제공부서,서울산업진흥원
1,작성자(책임자),120다산콜재단,생산일,2021-06-29
2,관리번호,D0000042894548,분류,경제


In [8]:
# 전치행렬 했을 때 column이 되게 하기 위해 set_index
table[[0, 1]].set_index(0)

,1
0,
원본시스템,다산콜센터
작성자(책임자),120다산콜재단
관리번호,D0000042894548


In [9]:
tb01 = table[[0, 1]].set_index(0).T
tb01

,원본시스템,작성자(책임자),관리번호
1,다산콜센터,120다산콜재단,D0000042894548


In [10]:
tb02 = table[[2, 3]].set_index(2).T
# 둘을 합치기 위해 index를 같게 설정
tb02.index = tb01.index
tb02

2,제공부서,생산일,분류
1,서울산업진흥원,2021-06-29,경제


In [11]:
# axis=1 옆으로, 열로써 붙임
pd.concat([tb01, tb02], axis=1)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제


In [12]:
# 위 과정을 진행하는 함수
def get_desc(response):
    table = pd.read_html(response.text)[-1]
    tb01 = table[[0, 1]].set_index(0).T
    tb02 = table[[2, 3]].set_index(2).T
    tb02.index = tb01.index
    df_desc = pd.concat([tb01, tb02], axis=1)
    return df_desc

get_desc(response)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제


# 내용 수집함수

In [13]:
def get_view_page(view_no):
    """
    1) url을 만든다.
    2) requests 로 요청을 보낸다.
    3) 분류를 수집하는 함수를 호출해서 분류 데이터프레임 반환
    4) 내용을 bs 으로 텍스트만 추출
    5) 3)번 내용에 내용, 내용번호를 함께 데이터프레임에 추가
    6) 반복문 대신 map 이나 apply 를 사용할 것이기 때문에 time.sleep 으로 쉬도록 함
    7) 5)번까지 수집한 내용을 반환
    """
    url = f"https://opengov.seoul.go.kr/civilappeal/view/?nid={view_no}"
    response = requests.get(url)
    df_desc = get_desc(response)
    html = bs(response.text)
    content = html.select("div.view-content.view-content-article > div > div.line-all")
    
    if content:
        content = content[0].get_text()
    else:
        content = ""
        
    df_desc["내용"] = content
    df_desc["내용번호"] = view_no
    
    # 반복문이 아닌 map 함수를 사용할 것이라서 함수 내부에 time.sleep
    time.sleep(0.01)
    return df_desc

In [14]:
# correct
view_no = 23194045
# error
# view_no = 22904492
get_view_page(view_no)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,내용번호
1,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...,23194045


# df에 map, apply 를 이용하요 일괄 적용
* map 은 Series 에서 쓰임
* apply 는 DataFrame과 Series 에서 쓰임

In [15]:
df

,번호,제목,생산일,조회수,내용번호
0,2304,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,126,26695536
1,2303,다자녀가정 실내 바닥매트 지원,2021-08-17,894,25670204
2,2302,[서울산업진흥원] 서울메이드란?,2021-06-29,5230,23194045
3,2301,마포 뇌병변장애인 비전센터,2021-03-12,1310,22477798
4,2300,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1626,21212235
5,2299,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1351,21095911
6,2298,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,1997,18955676
7,2297,"[종료사업] S-Taxi 앱 (에스택시, 서울택시승차앱) 시범운영",2019-06-03,2049,17949033
8,2296,장기요양 이동지원서비스(모두타 돌봄택시),2019-05-27,2800,17896364
9,2295,아름다운청년 전태일기념관,2019-03-20,1632,17505766


In [16]:
df["내용번호"]

0    26695536
1    25670204
2    23194045
3    22477798
4    21212235
5    21095911
6    18955676
7    17949033
8    17896364
9    17505766
Name: 내용번호, dtype: int64

In [45]:
tqdm.pandas()
# temp = df["내용번호"].map(lambda x: get_view_page(x))
# temp = df["내용번호"].map(get_view_page)

# tqdm.pandas() with progress_map 진행 상황을 알 수 있음
temp = df["내용번호"].progress_map(get_view_page)

  0%|          | 0/10 [00:00<?, ?it/s]

In [44]:
temp[8]

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,내용번호
1,다산콜센터,120다산콜재단,D0000036631298,서울특별시 도시교통실 교통기획관 택시물류과,2019-05-27,복지,\n 업무개요 보건복지부와 서울시택시운송사업조합의 민관협력사업으로 거동이...,17896364


In [53]:
df_view = pd.concat(temp.to_list()).reset_index(drop=True)
df_view

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,내용번호
0,다산콜센터,120다산콜재단,D0000046114132,대외협력과,2022-08-29,경제,\n ○ 하반기 운영일정 구 분 일 정 9월 10월 1...,26695536
1,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...,25670204
2,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...,23194045
3,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,2021-03-12,복지,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...,22477798
4,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,2020-09-16,복지,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...,21212235
5,다산콜센터,120다산콜재단,D0000040706778,서울특별시 자치구,2020-08-31,경제,"\n▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ...",21095911
6,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,2019-10-23,복지,\n서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰...,18955676
7,다산콜센터,120다산콜재단,D0000036784830,서울특별시 도시교통실 교통기획관 택시물류과,2019-06-03,교통,\r\n\t\t\t\t\t\t\t업무개요\n\n '빈차를 보고 택시를 이용' 하는 ...,17949033
8,다산콜센터,120다산콜재단,D0000036631298,서울특별시 도시교통실 교통기획관 택시물류과,2019-05-27,복지,\n 업무개요 보건복지부와 서울시택시운송사업조합의 민관협력사업으로 거동이...,17896364
9,다산콜센터,120다산콜재단,D0000035914975,서울특별시 노동민생정책관 노동정책담당관,2019-03-20,복지,\r\n\t\t\t\t\t\t\t구분\n\n내용\n\n\n \n\n주소\n\n 종로...,17505766


In [55]:
# 데이터 프레임 병합 방법 
# pd.concat([df, df_detail], axis=1)
# df_detail = pd.merge(df, df_view, on=["내용번호", "생산일"])
df_detail = df.merge(df_view, on=["내용번호", "생산일"])
df_detail

,번호,제목,생산일,조회수,내용번호,원본시스템,작성자(책임자),관리번호,제공부서,분류,내용
0,2304,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,126,26695536,다산콜센터,120다산콜재단,D0000046114132,대외협력과,경제,\n ○ 하반기 운영일정 구 분 일 정 9월 10월 1...
1,2303,다자녀가정 실내 바닥매트 지원,2021-08-17,894,25670204,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,복지,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...
2,2302,[서울산업진흥원] 서울메이드란?,2021-06-29,5230,23194045,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,경제,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...
3,2301,마포 뇌병변장애인 비전센터,2021-03-12,1310,22477798,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,복지,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...
4,2300,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1626,21212235,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,복지,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...
5,2299,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1351,21095911,다산콜센터,120다산콜재단,D0000040706778,서울특별시 자치구,경제,"\n▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ..."
6,2298,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,1997,18955676,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,복지,\n서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰...
7,2297,"[종료사업] S-Taxi 앱 (에스택시, 서울택시승차앱) 시범운영",2019-06-03,2049,17949033,다산콜센터,120다산콜재단,D0000036784830,서울특별시 도시교통실 교통기획관 택시물류과,교통,\r\n\t\t\t\t\t\t\t업무개요\n\n '빈차를 보고 택시를 이용' 하는 ...
8,2296,장기요양 이동지원서비스(모두타 돌봄택시),2019-05-27,2800,17896364,다산콜센터,120다산콜재단,D0000036631298,서울특별시 도시교통실 교통기획관 택시물류과,복지,\n 업무개요 보건복지부와 서울시택시운송사업조합의 민관협력사업으로 거동이...
9,2295,아름다운청년 전태일기념관,2019-03-20,1632,17505766,다산콜센터,120다산콜재단,D0000035914975,서울특별시 노동민생정책관 노동정책담당관,복지,\r\n\t\t\t\t\t\t\t구분\n\n내용\n\n\n \n\n주소\n\n 종로...


In [57]:
df = df_detail[["번호", "분류", "제목", "내용", "내용번호"]]

In [58]:
file_name = "seoul_120_sample.csv"

In [59]:
df.to_csv(file_name, index=False)

In [60]:
pd.read_csv(file_name)

,번호,분류,제목,내용,내용번호
0,2304,경제,하반기 농부의 시장 일정은 어떻게 되나요?,\n ○ 하반기 운영일정 구 분 일 정 9월 10월 1...,26695536
1,2303,복지,다자녀가정 실내 바닥매트 지원,\n 업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 ...,25670204
2,2302,경제,[서울산업진흥원] 서울메이드란?,\n 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일...,23194045
3,2301,복지,마포 뇌병변장애인 비전센터,\n ▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이...,22477798
4,2300,복지,위드유 서울 직장 성희롱.성폭력 예방센터,\n 업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하...,21212235
5,2299,경제,"[성동구] 반려동물 ""무료상담실"" 운영 안내","\n▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ...",21095911
6,2298,복지,서울시 [우리동네꼼꼼육아정보] 책자 발간,\n서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰...,18955676
7,2297,교통,"[종료사업] S-Taxi 앱 (에스택시, 서울택시승차앱) 시범운영",\r\n\t\t\t\t\t\t\t업무개요\n\n '빈차를 보고 택시를 이용' 하는 ...,17949033
8,2296,복지,장기요양 이동지원서비스(모두타 돌봄택시),\n 업무개요 보건복지부와 서울시택시운송사업조합의 민관협력사업으로 거동이...,17896364
9,2295,복지,아름다운청년 전태일기념관,\r\n\t\t\t\t\t\t\t구분\n\n내용\n\n\n \n\n주소\n\n 종로...,17505766
